<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/%20learning/operanalyst/linearprogcontinues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speciální úlohy lineárního programování

## Dopravní problém

https://www.youtube.com/watch?v=kFd8WgNwokQ

https://www.youtube.com/watch?v=Jr7LI-sUEmo

Přednáška z konference ESRI v roce 2019:

https://www.youtube.com/watch?v=DaB7J3rm_3M

https://econweb.ucsd.edu/~jsobel/172aw02/notes8.pdf

$$min \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij}x_{ij}$$
$$ \forall_{i=1,...,m} : \sum_{j=1}^{n} x_{ij} = S_i $$
$$ \forall_{j=1,...,n} : \sum_{i=1}^{m} x_{ij} = D_j $$

### Příklad

| $D_i \setminus S_i$ | $S_1$ | $S_2$ | $S_3$ | $S_4$ | $d_i$ |
|:--:|:--:|:--:|:--:|:--:|:--:|
| $D_1$ | 5 | 6 | 10 | 15 | 10 |
| $D_2$ | 7 | 6 | 3 | 10 | 10 |
| $D_3$ | 8 | 5 | 15 | 20 | 10 |
| $D_4$ | 10 | 20 | 30 | 30 | 10 |
| $s_j$ | 3 | 8 | 4 | 25 | 40 |

$$C=\begin{pmatrix} 
5 & 6 & 10 & 15\\
7 & 6 & 3 & 10\\
8 & 5 & 15 & 20\\
10 & 20 & 30 & 30
\end{pmatrix}$$
$$D=\begin{pmatrix}
10\\
10\\
10\\
10
\end{pmatrix}$$
$$S=\begin{pmatrix}
3\\
8\\
4\\
28
\end{pmatrix}$$

### Řešení v Python

#### Implementace Vogelovy aproximační metody

https://www.youtube.com/watch?v=HVHPBAKTZWw

In [5]:
from collections import defaultdict
 
def Vogel(costs, demand, supply):
    cols = sorted(demand.keys())
    res = dict((k, defaultdict(int)) for k in costs)
    g = {}
    for x in supply:
        g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g])
    for x in demand:
        g[x] = sorted(costs.keys(), key=lambda g: costs[g][x])

    while g:
        d = {}
        for x in demand:
            d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x]) if len(g[x]) > 1 else costs[g[x][0]][x]
        s = {}
        for x in supply:
            s[x] = (costs[x][g[x][1]] - costs[x][g[x][0]]) if len(g[x]) > 1 else costs[x][g[x][0]]
        f = max(d, key=lambda n: d[n])
        t = max(s, key=lambda n: s[n])
        t, f = (f, g[f][0]) if d[f] > s[t] else (g[t][0], t)
        v = min(supply[f], demand[t])
        res[f][t] += v
        demand[t] -= v
        if demand[t] == 0:
            for k, n in supply.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del demand[t]
        supply[f] -= v
        if supply[f] == 0:
            for k, n in demand.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del supply[f]
    return res

In [6]:
import numpy as np

C = {
    'd1':{'s1': 5, 's2': 6, 's3': 10, 's4': 15},
    'd2':{'s1': 7, 's2': 6, 's3': 3, 's4': 10},
    'd3':{'s1': 8, 's2': 5, 's3': 15, 's4': 20},
    'd4':{'s1': 10, 's2': 20, 's3': 30, 's4': 30},
     }
S = {'s1': 3, 's2': 8, 's3': 4, 's4': 28}
D = {'d1': 10, 'd2': 10, 'd3': 10, 'd4': 10}



In [7]:
res = Vogel({**C}, {**S}, {**D})
cols = sorted(D.keys())
for n in cols:
    print("\t", n, end='')
print()
cost = 0
for g in sorted(C):
    print(g, "\t", end='')
    for n in cols:
        y = res[g][n]
        if y != 0:
            print(y, end='')
        cost += y * C[g][n]
        print("\t", end='')
    print()
print("\n\nTotal Cost = ", cost)

IndexError: ignored